In [ ]:
reload("Utils")

In [1]:
using ProgressMeter
using SwiftObjectStores
using CorpusLoaders
using WordNet
using AdaGram
using AdaGramCompat
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Distances
using JLD

using SenseAlignment

importfrom(CorpusLoaders, :sensekey)
importfrom(WordNet, :sensekey)

In [3]:
const WN_PATH="data/corpora/WordNet-2.1/"
#WN_PATH3 = "/usr/share/nltk_data/corpora/wordnet/"
db = DB(WN_PATH)

WordNet.DB

In [4]:
#am = AdaGramCompat.AdaGramModel(load_model("models/adagram/v1_d100.adagram_model")...)
#am = AdaGramModel(load_model("models/adagram/more_senses.adagram_model")...)
#am = open(deserialize,"models/adagram/more_senses.adagram_model.jsz", "r");
am = load("models/adagram/more_senses.adagram_model.jld", "am");


In [ ]:
s_am = get_jld(SwiftService(), "sensemodels", "adagram/semhuff_more_senses.adagram_model.jld", "am");

In [ ]:
#ee = restore("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.jld");
ee = load("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.jld", "ee")

#ee = restore("models/ss/keep/tokenised_lowercase_WestburyLab.wikicorp.201004_50__m170000000.model");

In [5]:
challenges = load_challenges_semeval2007t7("data/corpora/wsd/semeval2007_t7/test/eng-coarse-all-words.xml");
solutions = load_solutions_semeval2007t7("data/corpora/wsd/semeval2007_t7/key/dataset21.test.key");
semcor = index_semcor(lazyload_semcor("data/corpora/semcor2.1/brown1/tagfiles/"));

In [6]:
function window(context, index::Int, window_size::Int=10)
    window_lower_bound = max(index - window_size÷2, 1)
    window_upper_bound = min(index + window_size÷2, length(context))
    view(context, [window_lower_bound:index-1 ; index+1:window_upper_bound])
end

function window(tagged_sense::TaggedSentence, index::Int, window_size::Int=10)
    context = lowercase.(strip_tags(tagged_sense))
    window(context, index, window_size)
end
   
function window(context, word::AbstractString, window_size::Int=10)
    context = lowercase.(context)
    occurances = find(context.==word)
    index =  length(occurances) > 0 ? occurances[ceil(Int, end/2)] : 0
    window(context, index, window_size)

end


window((drop(semcor,400) |> first |> last |> first)...)

window((synsets(db, db["fireplace", 'n']) |> first).gloss |> split, "fireplace")

10-element SubArray{String,1,Array{String,1},Tuple{Array{Int64,1}},false}:
 "fire"  
 "can"   
 "be"    
 "built;"
 "\"the" 
 "was"   
 "so"    
 "large" 
 "you"   
 "could" 

In [22]:

"Collect up the usages from a indexed tagged source"
function get_usages(usage_index::SemcorIndex, key)
    if haskey(usage_index, key)
        [window(lowercase.(strip_tags(sent)), index) for (sent, index) in usage_index[key]]
    else
        Vector{String}[]
    end
end

function get_usages(synset::Synset, lemma_word::AbstractString)
    gloss::Vector{SubString{String}} = lowercase.(punctuation_space_tokenize(synset.gloss))
    [window(gloss, lemma_word)]
end

function get_all_usages(wn::DB, semcor::SemcorIndex, lemma_word, pos)   
    lemma = db[pos, lemma_word]
    target_synsets::Vector{Synset} = synsets(db, lemma)
    
    Dict{Synset,AbstractVector{AbstractVector}}((synset => get_usages(synset, lemma_word)
        #[get_usages(semcor, sensekey(db, synset, lemma)); get_usages(synset, lemma_word)]
                    for synset in target_synsets))  
    
end

get_all_usages (generic function with 1 method)

In [23]:
all_identical(col) = length(col)==1 || !any(x->x!=col[1],col[2:end])

function _lexically_informed_embeddings(wn::DB,ee, word,lemma_word, pos)
    indexed_corpus = semcor #TODO: Pass this as a parameter, not as a gloel
    target_synset_examples = get_all_usages(wn, indexed_corpus, lemma_word, pos)
    target_synsets = collect(keys(target_synset_examples))
    lem = db[lemma_word, pos]
        
    embeddings = Vector{Vector{Float32}}(length(target_synsets))
    for (ii,(synset, examples)) in enumerate(target_synset_examples)
        context::Vector{SubString{String}} = vcat(examples...)
        embeddings[ii] =synthesize_embedding(ee,context, word, lemma_word)
    end
        
    
    if all_identical(embeddings)
        #Use MCWS
        score, index = findmax(sensecount(db, ss, lem ) for ss in target_synsets)
        target_synsets=[target_synsets[index]] # Throw out others
        embeddings=[embeddings[1]]
    end
        
    embeddings,target_synsets,lem
end

lexically_informed_embeddings(wn::DB, ee, lemma_word, pos) = lexically_informed_embeddings(db,ee, lemma_word, lemma_word, pos)
        
_li_embeddings = Dict{Tuple{DB, Any, String, String, Char}, Tuple{Vector{Vector{Float32}}, Vector{Synset}, Lemma}}()
function lexically_informed_embeddings(wn::DB, ee, word,lemma_word, pos)
    get!(_li_embeddings, (wn, ee, word, lemma_word, pos)) do
        _lexically_informed_embeddings(wn, ee, word, lemma_word, pos)
    end
end

lexically_informed_embeddings (generic function with 2 methods)

In [89]:
function supervised_wsd(challenge, ee, wn::DB)
    try
        embeddings,target_synsets,lem = lexically_informed_embeddings(wn,ee,
                                                            challenge.word, 
                                                            challenge.lemma, challenge.pos)
        
        priors = [float(sensecount(db, ss, lem)) for ss in target_synsets] #TODO use thing
        priors .+= 100
        #priors .+= sqrt(sum(priors))/length(priors)
        priors ./= sum(priors)
        probs_of_sense = general_wsd(ee, challenge.context, embeddings, priors)
               
     
        sense_index= indmax(probs_of_sense)        
        synset = target_synsets[sense_index]
        sk = sensekey(wn, synset, lem)
        Nullable(sk)
    catch ex
        isa(ex, KeyError) || rethrow(ex)
        @show ex
        Nullable{String}()
    end
end

function most_frequent_sense(challenge, wn::DB)
    try
        lem = wn[challenge.pos, challenge.lemma]
        target_synsets::Vector{Synset} = synsets(db, lem)
        
        sense_freqs =  Float32[sensecount(db, ss, lem) for ss in target_synsets]
        sense_index= indmax(sense_freqs)
        synset = target_synsets[sense_index]
        
        sk = sensekey(wn, synset,  lem)
        Nullable(sk)
    catch ex
        isa(ex, KeyError) || rethrow(ex)
        Nullable{String}()
    end
end



most_frequent_sense (generic function with 1 method)

In [90]:
function evalute_on_wsd_challenge(challenges, solutions, method)
    correct = 0
    incorrect = 0
    notattempted = 0
    @showprogress for (challenge, ground_solution) in zip(challenges, solutions)
        assert(challenge.id == ground_solution.id)
        output_sense = method(challenge)
        if isnull(output_sense)
            notattempted+=1
        else
            if get(output_sense) ∈ ground_solution.solutions
                correct+=1
            else
                incorrect+=1
            end
        end
    end
    @show notattempted
    precision = correct/(correct+incorrect)
    recall = correct/(correct+incorrect+notattempted)
    f1 = (2*precision*recall) / (precision+recall)
    return precision, recall, f1
end
    
    
function evalute_on_wsd_challenge(challenges, solutions, ee, wn::DB)
    method = challenge -> supervised_wsd(challenge, ee, wn)
    evalute_on_wsd_challenge(challenges, solutions, method)
end

function evalute_on_wsd_challenge_MFS(challenges, solutions, wn::DB)
    method = challenge -> most_frequent_sense(challenge, wn)
    evalute_on_wsd_challenge(challenges, solutions, method)
end

evalute_on_wsd_challenge_MFS (generic function with 1 method)

In [101]:
all_lemmas = collect(Base.flatten(values.(collect(values(db.lemmas)))));


In [114]:
all_sensecounts = [collect(values(sensecounts(db, lem))) for lem in all_lemmas]
total_cnts = sum.(all_sensecounts)
total_cnts = total_cnts[total_cnts.>0]  |> sort

24330-element Array{Int64,1}:
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
     1
    

In [118]:
total_cnts[end*98÷100]

97

In [94]:
counts[end*98÷100]

84

In [95]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, am, db))

#println("only nouns  : \t\t", 
#@show nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), am, db)
#println("only verbs  : \t\t", 
#@show vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), am, db)
#println("only adjecti: \t\t", 
#@show aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), am, db)
#println("only adverbs: \t\t", 
#@show rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), am, db)

Progress:  19%|████████                                 |  ETA: 0:00:08ex = KeyError(" SubString{String}[\"noncompetitively\"], nor SubString{String}[\"noncompetitively\"] have embeddings")
Progress:  23%|██████████                               |  ETA: 0:00:08ex = KeyError(" SubString{String}[\"semiliterate\"], nor SubString{String}[\"semiliterate\"] have embeddings")
Progress: 100%|█████████████████████████████████████████| Time: 0:00:09
notattempted = 2
overall: 		(0.7953242170269078,0.7946231820185103,0.794973544973545)


In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, s_am, db))

#println("only nouns  : \t\t", 
#@show nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), s_am, db)
#println("only verbs  : \t\t", 
#@show vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), s_am, db)
#println("only adjecti: \t\t", 
#@show aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), s_am, db)
#println("only adverbs: \t\t", 
#@show rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), s_am, db)

# "models/adagram/more_senses.adagram_model.jld
### with Adagram Disambig weighting (prior used)
### with SemCor data 
overall: 		(0.655408489274304,0.6328779197884531,0.6439461883408072)
- nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'),only_of_pos(solutions,'n'),am,db) = (0.7161410018552876,0.6967509025270758,0.706312900274474)
- vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'),only_of_pos(solutions,'v'),am,db) = (0.5106382978723404,0.4873096446700508,0.4987012987012987)
- aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'),only_of_pos(solutions,'a'),am,db) = (0.6619718309859155,0.649171270718232,0.6555090655509065)
- rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'),only_of_pos(solutions,'r'),am,db) = (0.7268041237113402,0.6778846153846154,0.7014925373134329)

### with Adagram Disambig weighting (prior used)
### with just glosses 

overall: 		(0.6704701049748973,0.6474217717055972,0.658744394618834)
- nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'),only_of_pos(solutions,'n'),am,db) = (0.7133580705009277,0.694043321299639,0.7035681610247027)
- vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'),only_of_pos(solutions,'v'),am,db) = (0.5780141843971631,0.5516074450084603,0.5645021645021645)
- aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'),only_of_pos(solutions,'a'),am,db) = (0.6535211267605634,0.6408839779005525,0.6471408647140865)
- rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'),only_of_pos(solutions,'r'),am,db) = (0.7319587628865979,0.6826923076923077,0.7064676616915424)



# "semhuff_more_senses.adagram_model.jld

### with Adagram Disambig weighting (prior used)
### with just glosses 
 - overall: 		(0.6704701049748973,0.6474217717055972,0.658744394618834)
 - nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'),only_of_pos(solutions,'n'),am,db) = (0.7133580705009277,0.694043321299639,0.7035681610247027)
 - vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'),only_of_pos(solutions,'v'),am,db) = (0.5780141843971631,0.5516074450084603,0.5645021645021645)
 - aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'),only_of_pos(solutions,'a'),am,db) = (0.6535211267605634,0.6408839779005525,0.6471408647140865)
 - rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'),only_of_pos(solutions,'r'),am,db) = (0.7319587628865979,0.6826923076923077,0.7064676616915424)
 
 ### with Adagram Disambig weighting (prior used)
 ### With semcore
 - overall: 		(0.6134185303514377,0.5923314235345968,0.6026905829596412)
 - nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'),only_of_pos(solutions,'n'),s_am,db) = (0.6252319109461967,0.6083032490974729,0.6166514181152789)
 - vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'),only_of_pos(solutions,'v'),s_am,db) = (0.5638297872340425,0.5380710659898477,0.5506493506493505)
 - aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'),only_of_pos(solutions,'a'),s_am,db) = (0.6169014084507042,0.6049723756906077,0.610878661087866)
 - rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'),only_of_pos(solutions,'r'),s_am,db) = (0.6855670103092784,0.6394230769230769,0.6616915422885572)
 
 
 # Most Frequent Sense
 
  -  overall: 		(0.7783164389598942,0.7783164389598942,0.7783164389598942)
  - only nouns  : 		(0.7653429602888087,0.7653429602888087,0.7653429602888087)
  - only verbs  : 		(0.7529610829103215,0.7529610829103215,0.7529610829103215)
  - only adjecti: 		(0.8038674033149171,0.8038674033149171,0.8038674033149171)
  - only adverbs: 		(0.875,0.875,0.875)


In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, ee, db))

#println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), ee, db))
#println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), ee, db))
#println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), ee, db))
#println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), ee, db))

In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge_MFS(challenges, solutions, db))

#println("only nouns  : \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), db))
#println("only verbs  : \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), db))
#println("only adjecti: \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), db))
#println("only adverbs: \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), db))


In [ ]:
#Zero shot WSI for 1 shot WSD
using Training

In [ ]:
rr = FixedWordSenseEmbedding(ee.dimension, random_inited, huffman_tree; initial_nsenses=50)
rr.corpus_size = ee.corpus_size
rr.distribution = ee.distribution
rr.codebook = ee.codebook
rr.classification_tree = ee.classification_tree
Training.initialize_embedding(rr);

In [ ]:
rr.embedding["us"] |> length

In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, rr, db))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), rr, db))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), rr, db))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), rr, db))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), rr, db))

In [ ]:
hh = deepcopy(rr)
for word in keys(rr.embedding)
    append!(hh.embedding[word], ee.embedding[word])
end

In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, hh, db))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), hh, db))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), hh, db))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), hh, db))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), hh, db))

In [ ]:
mapping_corpus = load_semcor("data/corpora/semcor2.1/brown1/tagfiles/");

In [ ]:
function agirresAlignment(ee, wn::DB, mapping_corpus, hard=true)
    maps = Dict{Tuple{String, Char}, Matrix{Float32}}()
    freqs = Dict{Tuple{String, Char}, Vector{Int}}()
    
    function proc_word(word::TaggedWord, sentence::TaggedSentence)
    end
    
    function proc_word(taggedword::SenseAnnotatedWord, sentence::TaggedSentence)
        local wn_sensekeys
        pos = pennPOStoWordNetPOS(taggedword.pos)
        

        
        try 
            wn_sensekeys = sensekeys(wn, wn[pos, taggedword.lemma])
        catch exception
            exception |> typeof <: KeyError || rethrow()
            warn("Could not find wordnet lemma for $taggedword")
            return
        end
        
        target_wnsn = findfirst(wn_sensekeys .== sensekey(taggedword))
        if target_wnsn==0
            warn("$(sensekey(taggedword)) not in  $(wn_sensekeys).\n\nIssue is with $taggedword")
            return
        end

        wvs = all_word_sense_vectors(ee,taggedword.word)
        if length(wvs) == 0
            warn("No embedding for $(taggedword.word); skipping")
            return
        end
        
        ########
        
        map = get!(maps, (taggedword.lemma, pos)) do
            zeros(length(wvs), length(wn_sensekeys))
        end
        
        freq = get!(freqs, (taggedword.lemma, pos)) do 
            zeros(Int, length(wn_sensekeys))
        end
        
        @assert(length(freq) == size(map,2) == length(wn_sensekeys),
        """
        $(size(map)) $(length(freq)) $(length(wn_sensekeys))
        $taggedword
        """ )
        @assert(size(map,1) == length(wvs))
        
        
        context = lowercase.(strip_tags(sentence))
        
        wv_probs = general_wsd(ee,context, wvs)
        @assert !any(isnan.(wv_probs))
        freq[target_wnsn] += 1
        if hard
            map[indmax(wv_probs),target_wnsn] += 1
        else
            map[:,target_wnsn] += wv_probs
        end
        
    end
    
    for sentence in mapping_corpus
        for word in sentence
            proc_word(word, sentence)
        end
    end
 
    if !(hard)
        for ((word,pos), freq) in freqs
            map = maps[(word,pos)]
            map ./= freq'
            map[isnan.(map)] = 0.0 #NaNs are just frequency 0 items
            @assert(all((isapprox(x,1.0; atol=10.0^-5)  for x in sum(map, 1))))
        end
    end
    
    maps, freqs
end

In [ ]:
maps, freqs  = agirresAlignment(am, db, mapping_corpus, false);

In [ ]:
m = maps["city",'n']
f = freqs["city",'n']
@show m
@show f

In [ ]:
isapprox(1.0,2.0; atol=10.0)

In [ ]:
a = [1.0 2 3 0 ; 11 22 33 0 ; 111 222 333 0; 1111 2222 3333 0; 11111 22222 33333 5]
b = [5, -5, 1, 0]

@show a./b'

a./=b'
a[isnan(a)]=0.0
a

In [ ]:
n = copy(m)
@show n
@show f
n./=f'
n[isnan.(n)]=0.0
n

In [ ]:
isnan.([NaN 1; Inf 4])

In [ ]:
ww = CorpusLoaders.Semcor.SenseAnnotatedWord{SubString{String}}("NN","funds",1,"1:21:00::","funds")
sensekey(ww)

In [ ]:
@show ww.pos 
ww.lemma

In [ ]:
methods(sensekey)